# 📊 Crypto Trading Bot Data Analysis
This notebook loads historical OHLCV data from Binance, uses a trained model (`model_v1.pkl`), and visualizes important features.

In [ ]:
# Imports
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import joblib
import numpy as np
from sklearn.preprocessing import StandardScaler
from src.collectors.fetch_historical import fetch_ohlcv
from src.utils.features import generate_features

In [ ]:
# Load historical OHLCV data
symbol = "BTCUSDT"
interval = "15m"
limit = 500
df = fetch_ohlcv(symbol=symbol, interval=interval, limit=limit)
df.tail()

In [ ]:
# Generate technical indicators and features
features_df = generate_features(df.copy())
features_df.dropna(inplace=True)
features_df.tail()

In [ ]:
# Visualize OHLCV trends
plt.figure(figsize=(15, 5))
plt.plot(df.index, df['close'], label='Close Price')
plt.title(f'{symbol} Closing Price - {interval}')
plt.xlabel('Time')
plt.ylabel('Price (USDT)')
plt.legend()
plt.grid()
plt.show()

In [ ]:
# Load trained model
model = joblib.load('models/model_v1.pkl')

In [ ]:
# Predict with model
X = features_df.drop(columns=['target']) if 'target' in features_df.columns else features_df.copy()
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)
predictions = model.predict(X_scaled)
features_df['prediction'] = predictions
features_df[['prediction']].tail()

In [ ]:
# Plot predictions
plt.figure(figsize=(15, 5))
plt.plot(features_df.index, df.loc[features_df.index]['close'], label='Close Price')
buy_signals = features_df[features_df['prediction'] == 1]
sell_signals = features_df[features_df['prediction'] == -1]

plt.scatter(buy_signals.index, df.loc[buy_signals.index]['close'], marker='^', color='green', label='Buy Signal')
plt.scatter(sell_signals.index, df.loc[sell_signals.index]['close'], marker='v', color='red', label='Sell Signal')
plt.title(f'{symbol} Model Predictions')
plt.legend()
plt.grid()
plt.show()